In [4]:
import click
import os
import numpy as np
import torch
from torch.utils.data import DataLoader

from tqdm import tqdm
from util import configure_device, get_dataset
from models.split_vae import VAE
import matplotlib
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

@click.group()
def cli():
    pass

def extract(
    orig_img_id = "00084",
    root=None,
    vae_chkpt_path="/home/bias-team/Mo_Projects/DiffuseVAE/logs/vae_carla_day/checkpoints/vae-carla_day-epoch=1499-train_loss=0.0000.ckpt",
    device="gpu:0",
    dataset_name="carla",
    image_size=128,
    save_path=os.getcwd(),
):
    # dev, _ = configure_device(device)
    dev = 'cuda'
    root = os.path.join(root, orig_img_id)
    # Dataset
    dataset = get_dataset(dataset_name, root, image_size, norm=False, flip=False)

    # Loader
    loader = DataLoader(
        dataset,
        1,
        num_workers=1,
        pin_memory=True,
        shuffle=False,
        drop_last=False,
    )

    # Load VAE
    vae = VAE.load_from_checkpoint(vae_chkpt_path, input_res=image_size).to(dev)
    vae.eval()

    z_list = []
    for _, batch in tqdm(enumerate(loader)):
        batch = batch.to(dev)
        with torch.no_grad():

            batch_aux, _ = vae.scramble(batch)
            mu, logvar = vae.encode(batch)
            mu_aux, logvar_aux = vae.encode_aux(batch_aux)

            z_main = vae.reparameterize(mu, logvar)
            z_aux = vae.reparameterize(mu_aux, logvar_aux)

            z_main = torch.cat((mu,mu_aux), dim=1)
            # z_main = mu_aux
            # z_main = mu
        # Not transferring to CPU leads to memory overflow in GPU!
        z_list.append(z_main.cpu().detach().numpy())
    z_arr = np.array(z_list)
    image_labels = len(z_list)*[orig_img_id]
    np.save(os.path.join(root, f"{orig_img_id}_split.npy"), z_arr)
    return z_arr, image_labels


def get_latent(data_path, vae_chkpt_path):
    img_id_list = os.listdir(data_path)

    for i, img_id in enumerate(img_id_list):
        z, image_labels = extract(orig_img_id=img_id, root=data_path, vae_chkpt_path=vae_chkpt_path)
        z = np.squeeze(z)
        if len(z.shape) == 1:
            z = np.expand_dims(z, axis=0)        
        if i == 0:
            z_total = z
            image_labels_total = image_labels
        else:
            z_total = np.concatenate((z_total, z), axis=0)
            image_labels_total += image_labels
    labels, colors = get_class_map(image_labels_total, img_id_list)
    return z_total, labels

def get_class_map(image_labels_total, img_id_list):
    # Create Mapping function from img_id to class 0,1,2,3,...
    img_class_map = {}
    colors = []
    labels = []
    for i, img_id in enumerate(img_id_list):
        img_class_map[img_id] = i
        colors.append(i)

    for i in image_labels_total:
        temp = img_class_map[i]
        labels.append(temp)
    labels = np.array(labels)
    return labels, colors



        

In [5]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
## CARLA
# test_dir = "/home/bias-team/Mo_Projects/DiffuseVAE/ood_experiment/carla2_ood/tests"
test_dir = "/home/bias-team/Mo_Projects/DiffuseVAE/ood_experiment/carla2_ood/tests2"
test_name_list = os.listdir(test_dir)
train_score_dict = {}
test_score_dict = {}
for test_name in test_name_list:
    train_score_list = []
    test_score_list = []
    test_path = os.path.join(test_dir, test_name)
    for seed in range(1,11):
        vae_ckpt_path =  f"/home/bias-team/Mo_Projects/DiffuseVAE/logs/vae_carla_day{seed}/checkpoints/vae-carla_day-epoch=1499-train_loss=0.0000.ckpt"
        x_train, y_train = get_latent("/home/bias-team/Mo_Projects/DiffuseVAE/ood_experiment/carla2_ood/train",
                                    vae_ckpt_path)
        x_test, y_test = get_latent(test_path,
                                    vae_ckpt_path)

        clf =  make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=1))
        clf.fit(x_train, y_train)
        y_pred_train = clf.predict(x_train)
        y_pred_test = clf.predict(x_test)
        train_score = accuracy_score(y_train, y_pred_train)
        test_score = accuracy_score(y_test, y_pred_test)
        train_score_list.append(train_score)
        test_score_list.append(test_score)
        print(f"{test_name} seed {seed}")
        print("train:", train_score)
        print("test:", test_score)
    train_score_dict[test_name] = train_score_list
    test_score_dict[test_name] = test_score_list


1it [00:00, 15.99it/s]
1it [00:00, 17.10it/s]
1it [00:00, 17.55it/s]
1it [00:00, 16.56it/s]
1it [00:00, 16.85it/s]
1it [00:00, 17.47it/s]
1it [00:00, 16.98it/s]
1it [00:00, 17.28it/s]
1it [00:00, 19.06it/s]
1it [00:00, 16.74it/s]
10it [00:00, 68.84it/s]
10it [00:00, 70.27it/s]
10it [00:00, 71.02it/s]
10it [00:00, 71.89it/s]
10it [00:00, 71.48it/s]
10it [00:00, 73.07it/s]
10it [00:00, 71.86it/s]
10it [00:00, 71.71it/s]
10it [00:00, 71.17it/s]
10it [00:00, 71.91it/s]


test_flare2 seed 1
train: 1.0
test: 0.63


1it [00:00, 15.31it/s]
1it [00:00, 15.19it/s]
1it [00:00, 17.15it/s]
1it [00:00, 17.79it/s]
1it [00:00, 17.88it/s]
1it [00:00, 17.23it/s]
1it [00:00, 17.34it/s]
1it [00:00, 17.39it/s]
1it [00:00, 17.28it/s]
1it [00:00, 17.09it/s]
10it [00:00, 69.44it/s]
10it [00:00, 68.13it/s]
10it [00:00, 67.40it/s]
10it [00:00, 63.54it/s]
10it [00:00, 64.24it/s]
10it [00:00, 70.11it/s]
10it [00:00, 60.42it/s]
10it [00:00, 66.98it/s]
10it [00:00, 67.60it/s]
10it [00:00, 56.13it/s]


test_flare2 seed 2
train: 1.0
test: 0.65


1it [00:00, 16.51it/s]
1it [00:00, 17.59it/s]
1it [00:00, 15.00it/s]
1it [00:00, 15.21it/s]
1it [00:00, 17.27it/s]
1it [00:00, 17.23it/s]
1it [00:00, 18.41it/s]
1it [00:00, 12.83it/s]
1it [00:00, 12.61it/s]
1it [00:00, 18.87it/s]
10it [00:00, 69.37it/s]
10it [00:00, 70.19it/s]
10it [00:00, 62.82it/s]
10it [00:00, 61.92it/s]
10it [00:00, 65.50it/s]
10it [00:00, 64.94it/s]
10it [00:00, 63.02it/s]
10it [00:00, 56.16it/s]
10it [00:00, 58.51it/s]
10it [00:00, 57.12it/s]


...

test_flare2 seed 8
train: 1.0
test: 0.59


1it [00:00, 15.75it/s]
1it [00:00, 14.72it/s]
1it [00:00, 16.03it/s]
1it [00:00, 14.22it/s]
1it [00:00, 16.68it/s]
1it [00:00, 16.02it/s]
1it [00:00, 12.81it/s]
1it [00:00, 16.38it/s]
1it [00:00, 15.96it/s]
1it [00:00, 14.54it/s]
10it [00:00, 55.97it/s]
10it [00:00, 62.64it/s]
10it [00:00, 53.35it/s]
10it [00:00, 57.65it/s]
10it [00:00, 53.16it/s]
10it [00:00, 65.36it/s]
10it [00:00, 56.75it/s]
10it [00:00, 59.55it/s]
10it [00:00, 59.24it/s]
10it [00:00, 56.89it/s]


test_flare2 seed 9
train: 1.0
test: 0.36


1it [00:00, 13.01it/s]
1it [00:00, 16.30it/s]
1it [00:00, 14.27it/s]
1it [00:00, 13.65it/s]
1it [00:00, 15.78it/s]
1it [00:00, 12.50it/s]
1it [00:00, 14.66it/s]
1it [00:00, 17.12it/s]
1it [00:00, 14.02it/s]
1it [00:00, 15.82it/s]
10it [00:00, 58.20it/s]
10it [00:00, 48.14it/s]
10it [00:00, 58.78it/s]
10it [00:00, 64.14it/s]
10it [00:00, 52.07it/s]
10it [00:00, 63.45it/s]
10it [00:00, 54.69it/s]
10it [00:00, 59.50it/s]
10it [00:00, 55.09it/s]
10it [00:00, 60.98it/s]

test_flare2 seed 10
train: 1.0
test: 0.54


In [7]:
# # print(test_score_dict)
# rain_acc, rain_sd = np.mean(test_score_dict['test_rain']), np.std(test_score_dict['test_rain'])
# fog_acc, fog_sd = np.mean(test_score_dict['test_fog']), np.std(test_score_dict['test_fog'])
# flare_acc, flare_sd = np.mean(test_score_dict['test_flare']), np.std(test_score_dict['test_flare'])
# snow_acc, snow_sd = np.mean(test_score_dict['test_snow']), np.std(test_score_dict['test_snow'])
# shadow_acc, shadow_sd = np.mean(test_score_dict['test_shadow']), np.std(test_score_dict['test_shadow'])

# print(f"Rain: {rain_acc} +- {rain_sd}")
# print(f"Fog: {fog_acc} +- {fog_sd}")
# print(f"Flare: {flare_acc} +- {flare_sd}")
# print(f"Snow: {snow_acc} +- {snow_sd}")
# print(f"Shadow: {shadow_acc} +- {shadow_sd}")
flare2_acc, flare2_sd = np.mean(test_score_dict['test_flare2']), np.std(test_score_dict['test_flare2'])
print(f"Flare2: {flare2_acc} +- {flare2_sd}")

Flare2: 0.541 +- 0.08129575634681063
